In [8]:
# !pip install dataset
# !pip install setfit

# # skew towards false postive is good

In [3]:
import pandas as pd
import pyarrow as pa
from sklearn.model_selection import train_test_split
from datasets import Dataset


# Specify the path to your Excel file
file_path = '/content/companydata.xlsx'

# Read the Excel file into a DataFrame
df = pd.read_excel(file_path)

# Split the DataFrame into training and temporary sets
train_df, temp_df = train_test_split(df, test_size=0.4, random_state=42)

# Split the temporary set into testing and validation sets
test_df, val_df = train_test_split(temp_df, test_size=0.5, random_state=42)


train_dataset = Dataset.from_pandas(train_df[:30])
test_dataset = Dataset.from_pandas(test_df[:10])
eval_dataset = Dataset.from_pandas(val_df[:10])



In [4]:
train_dataset

Dataset({
    features: ['description', 'label', '__index_level_0__'],
    num_rows: 30
})

In [5]:
# from datasets import load_dataset
from setfit import SetFitModel, Trainer, TrainingArguments, sample_dataset
import os
os.environ['WANDB_DISABLED'] = 'true'


# Load a SetFit model from Hub
model = SetFitModel.from_pretrained(
    # "sentence-transformers/paraphrase-mpnet-base-v2",
    # "tyzp-INC/few-shot-multilingual-e5-large-xnli",
    # "thenlper/gte_small",
    "moshew/bge-small-en-v1.5_setfit-sst2-english",
    # "BAAI/bge-small-en-v1.5",
    # labels=["negative", "positive"],
    labels=[0,1,2],
)

args = TrainingArguments(
    batch_size=8,
    num_epochs=1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    metric="accuracy",
    column_mapping={"description": "text", "label": "label"}  # Map dataset columns to text/label expected by trainer
)

# Train and evaluate
trainer.train()
metrics = trainer.evaluate(test_dataset)
print(metrics)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/748 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

...

***** Running training *****
  Num unique pairs = 768
  Batch size = 8
  Num epochs = 1
  Total optimization steps = 96


Epoch,Training Loss,Validation Loss,Embedding Loss,Rate
1,No log,No log,0.490100,0.000000


  0%|          | 0/10 [00:00<?, ?it/s]

Loading best SentenceTransformer model from step 96.
Applying column mapping to the evaluation dataset
***** Running evaluation *****


{'accuracy': 0.9}


In [6]:
preds = model(["Cohesive helps small business owners save time and money. We offer technology-enabled virtual assistance services.  Check us out at www.cohesiveapp.com!"])
preds

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[0]

In [14]:
test_sentences =test_df[:10]['description'].tolist()
test_sentences_y =test_df[:10]['label'].tolist()

In [16]:
pip install torcheval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 5.4 MB/s eta 0:00:00


In [23]:
test_sentences_y

[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]

In [24]:
model(test_sentences)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [21]:
import torch

def f1_score_pytorch(predictions, targets):
    predictions = torch.tensor(predictions)
    targets = torch.tensor(targets)

    # Calculate true positives, false positives, and false negatives
    tp = torch.sum(predictions * targets)
    fp = torch.sum(predictions * (1 - targets))
    fn = torch.sum((1 - predictions) * targets)

    # Calculate precision and recall
    precision = tp / (tp + fp + 1e-10)
    recall = tp / (tp + fn + 1e-10)

    # Calculate F1 score
    f1 = 2 * (precision * recall) / (precision + recall + 1e-10)

    return f1.item(),precision,recall

In [22]:
f1_score_pytorch(model(test_sentences), test_sentences_y)

(0.0, tensor(0.), tensor(0.))

In [17]:
from torcheval.metrics.functional import multiclass_f1_score

multiclass_f1_score(model(test_sentences), test_sentences_y, num_classes=2, average=None)

AttributeError: 'list' object has no attribute 'size'